In [ ]:
#pip install kaggle --upgrade

In [ ]:
#!export KAGGLE_USERNAME=mohamedmalhou
#!export KAGGLE_KEY=77cb5e2e17b8d6b9ab875c1d636404b8

In [ ]:
#!kaggle competitions download airbus-ship-detection

In [ ]:
#!rm airbus-ship-detection.zip

In [ ]:
#!kaggle competitions list -s airBus

In [ ]:
#!jupyter notebook --NotebookApp.iopub_data_rate_limit=100000000

In [ ]:
import os
import sys
import random
import warnings

import numpy as np
from skimage.transform import resize
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.nn.functional as F

import pandas as pd

%matplotlib inline

from tqdm import tqdm_notebook, tnrange
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from skimage.feature import canny
from skimage.filters import sobel,threshold_otsu, threshold_niblack,threshold_sauvola
from skimage.segmentation import felzenszwalb, slic, quickshift, watershed
from skimage.segmentation import mark_boundaries
from scipy import signal

import cv2
from PIL import Image
import pdb
from tqdm import tqdm
import seaborn as sns
import os 
from glob import glob

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from os import listdir
from os.path import isfile, join
mypath = ""
size = 768
import PIL



In [ ]:
df = pd.read_csv(mypath + "train_ship_segmentations_v2.csv", index_col=0)

In [ ]:
df = df.reset_index()
df['ship_count'] = df.groupby('ImageId')['ImageId'].transform('count')
df.loc[df['EncodedPixels'].isnull().values,'ship_count'] = 0  #see infocusp's comment

In [ ]:

sns.set_style("white")
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.distplot(df['ship_count'],kde=False)
plt.title('Ship Count Distribution in Train Set')

print(df['ship_count'].describe())

In [ ]:
display(df.head())


In [ ]:

train_ids = df.ImageId.values
df = df.set_index('ImageId')

In [ ]:
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)
def masks_as_image(in_mask_list, all_masks=None,size = 768):
    # Take the individual ship masks and create a single mask array for all ships
    if all_masks is None:
        all_masks = np.zeros((size, size), dtype = np.int16)
    #if isinstance(in_mask_list, list):
    for mask in in_mask_list:
        if isinstance(mask, str):
            all_masks += rle_decode(mask)
    return np.expand_dims(all_masks, -1)
def rle_decode(mask_rle, shape=(768, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    print(mask_rle)
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction
def rle_decode_crop(mask_rle, shape=(768, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    size = 768
    X,Y = 0,0
    count = 0
    for lo, hi in zip(starts, ends):
        count+=(hi-lo)
        i,j = lo//size, lo%size
        X+=i*(hi-lo)
        Y+=j*(hi-lo) + (hi-lo+1)*(hi-lo)//2
        img[lo:hi] = 1
        
    return (img.reshape(shape).T,X//count,Y//count)  # Needed to align to RLE direction
def masks_as_image_crop(in_mask_list, all_masks=None,size = 768):
    # Take the individual ship masks and create a single mask array for all ships
    if all_masks is None:
        all_masks = np.zeros((size, size), dtype = np.int16)
    #if isinstance(in_mask_list, list):
    
    centers = []
    for mask in in_mask_list:
        if isinstance(mask, str):
            Mask,i,j = rle_decode_crop(mask)
            all_masks += Mask
            centers.append((i,j))
   
    return (np.expand_dims(all_masks, -1), centers)
def dist(i,j,x,y):
    return ((i-x)**2 + (j-y)**2)**0.5

In [ ]:

mypath = ""
trainfiles = listdir(  "train_v2") 
print(len(trainfiles))

testfiles = listdir("test_v2")


In [ ]:
load_img = lambda filename: np.array(PIL.Image.open(f"train_v2/{filename}"))

In [ ]:
"""
#split image into 4 by 4
trdata = []
i = 0

for filename in trainfiles[60000:120000] : 
  if i%10000 == 0:
    print("iteration ",i)
  i+=1
  if df.query('ImageId=="'+filename+'"')['ship_count'][0] >=1 :
    im = load_img(filename)
    ma = masks_as_image(df.query('ImageId=="'+filename+'"')['EncodedPixels'])
    for k in range(4):
        for l in range(4):
            trdata.append((im[224*k - 43*k: 224*(k+1)-43*k, 224*l - 43*l: 224*(l+1) - 43*l],
                           ma[224*k - 43*k: 224*(k+1)-43*k, 224*l - 43*l: 224*(l+1) - 43*l]))
    #trdata.append((resize(load_img(filename),(224,224)),
                  #resize(masks_as_image(df.query('ImageId=="'+filename+'"')['EncodedPixels']),(224,224))))
                  
tsdata = []

for filename in trainfiles[120000:122000] : 
  if i%1000 == 0:
    print("iteration ",i)
  i+=1
  if df.query('ImageId=="'+filename+'"')['ship_count'][0] >=1 :
    tsdata.append((resize(load_img(filename),(224,224)),
                   resize(masks_as_image(df.query('ImageId=="'+filename+'"')['EncodedPixels']),(224,224))))
"""
trdata = []
Original = []
og = 768
s = 224
counter = 0
for filename in trainfiles[180000:189000] : 

  counter+=1
  if counter%1000 ==0:
        print("iteration ", counter)
  if df.query('ImageId=="'+filename+'"')['ship_count'][0] >=1 :
    im = load_img(filename)
    ma,centers = masks_as_image_crop(df.query('ImageId=="'+filename+'"')['EncodedPixels'])
    for k in range(len(centers)):
        j,i = centers[k]
        intersects = False
        for m,n in centers[:k]:
            if dist(j,i,m,n) <56 :
                intersects = True
        if not intersects :
            trdata.append((im[max(i-s//2,0)-max(i+s//2-og,0):min(og,i+s//2) + max(0,s//2-i),
                          max(j-s//2,0)-max(j+s//2-og,0):min(og,j+s//2) + max(0,s//2-j)],
                       ma[max(i-s//2,0)-max(i+s//2-og,0):min(og,i+s//2) + max(0,s//2-i),
                          max(j-s//2,0)-max(j+s//2-og,0):min(og,j+s//2) + max(0,s//2-j)]))
    #Original.append((im,ma))

In [ ]:

def prepare_dataset(x,size = 224):
  num_examples = len(x)
  
  Y = np.zeros((num_examples, size, size, 1), dtype=np.int32)
  X = np.zeros((num_examples, size, size, 3), dtype=np.int32)
  for i in range(num_examples):
    im,pixels = x[i]
    Y[i,...] = pixels
    X[i,...] = im
  print("X shape is ",X.shape)
  return X, Y
def prepare_dataloader(X_train, y_train, batch_size):
  

  trainset = torch.utils.data.TensorDataset(torch.tensor(np.transpose(X_train, (0, 3, 1, 2))),
                                            torch.tensor(y_train[..., 0]))
 
  trainloader = torch.utils.data.DataLoader(trainset,batch_size=batch_size,shuffle=True,num_workers=2)

  return trainloader
#print(len(trdata))

In [ ]:
trainloader = prepare_dataloader(*prepare_dataset(trdata),16)


In [ ]:
testloader = prepare_dataloader(*prepare_dataset(tsdata),16)


In [ ]:

class ResNetBlock(nn.Module):
  def __init__(self, in_planes, planes, stride=1,id = 1):
    super(ResNetBlock, self).__init__()
    self.conv1 = nn.Conv2d(
        in_planes, planes, kernel_size=3, stride=stride, padding=1)
    self.bn1 = nn.BatchNorm2d(planes)
    self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1)
    self.bn2 = nn.BatchNorm2d(planes)

    self._id_ = nn.Sequential()
    if stride != 1:
      self._id_ = nn.Conv2d(
          in_planes, planes, kernel_size=3, stride=2, padding=1)
    if id != 1:
      self._id_ = nn.Conv2d(
          in_planes, planes, kernel_size=1)

  def forward(self, x):
      #print("x is ", x.size())
      x_id = self._id_(x)
      #print("id x is ", x_id.size())
      bx = F.relu(self.bn1(self.conv1(x)))
      #print("bx is ",bx.size())
      bx = F.relu(self.bn2(self.conv2(bx)) + x_id)

      return bx


class U_ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_filters=16, input_dim=1, image_size=64):
        super(U_ResNet, self).__init__()
        self.in_planes = num_filters
        self.image_size = image_size

        self.conv1 = nn.Conv2d(input_dim, num_filters,
                               kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(num_filters)
        self.relu = nn.ReLU()
        self.R1 = self._make_layer(block, self.in_planes, 2, 1,1)
        self.bn2 = nn.BatchNorm2d(self.in_planes)
        self.id1 = nn.Sequential()
        self.R2 = self._make_layer(block, 2*self.in_planes, 2, 2,1)
        self.in_planes*=2
        self.bn3 = nn.BatchNorm2d(self.in_planes)
        self.R3 = self._make_layer(block, 2*self.in_planes, 2, 2,1)
        self.in_planes*=2
        self.bn4 = nn.BatchNorm2d(self.in_planes)
        self.deconv1 = nn.ConvTranspose2d(self.in_planes, self.in_planes//2, 2, stride=2)
        
        self.R4 = self._make_layer(block, self.in_planes//2, num_blocks[0], 1,2)
        self.in_planes//=2
        self.deconv2 = nn.ConvTranspose2d(self.in_planes, self.in_planes//2, 2, stride=2)
        self.bn5 = nn.BatchNorm2d(self.in_planes//2)
        self.R5 = self._make_layer(block, self.in_planes//2, num_blocks[1], 1,2)
        self.bn6 = nn.BatchNorm2d(num_filters)
        self.conv2 = nn.Conv2d(num_filters, 2, kernel_size=1)

    def _make_layer(self, block, planes, num_blocks, stride,_id):
        layers = []
        layers.append(block(self.in_planes, planes, stride,_id))
        for _ in range(num_blocks-1):
          layers.append(block(planes, planes, 1))
        return nn.Sequential(*layers)

    def forward(self, x):

        out = self.relu(self.bn1(self.conv1(x)))
       
        out = self.bn2(self.R1(out))
        
        concat1 = self.id1(out)
       
        out = self.bn3(self.R2(out))
       
        concat2 = self.id1(out)
        
        out = self.bn4(self.R3(out))
       
        out = self.deconv1(out)
        
        #print(concat2.size(),out.size())
        out = self.R4(torch.cat((concat2, out), dim=1))
       
        out = self.bn5(self.deconv2(out))
        #print("deconv2" , out.size())
        #print("id2 " , concat1.size())
   
        out = torch.cat((concat1, out), dim=1)
        #print("concat ", out.size())
      
        out = self.bn6(self.R5(out))
       
        #print("out of R5 is " , out.size() )
        out = self.conv2(out)
      
        return out

def U_resNet():
    return U_ResNet(ResNetBlock, [2, 2],input_dim=3,image_size=768)


In [ ]:
class LossBinary:
    """
     Implementation from  https://github.com/ternaus/robot-surgery-segmentation
    """

    def __init__(self, jaccard_weight=0):
        self.nll_loss = nn.BCEWithLogitsLoss()
        self.jaccard_weight = jaccard_weight

    def __call__(self, outputs, targets):
        loss = self.nll_loss(outputs, targets)

        if self.jaccard_weight:
            eps = 1e-15
            jaccard_target = (targets == 1.0).float()
            jaccard_output = F.sigmoid(outputs)

            intersection = (jaccard_output * jaccard_target).sum()
            union = jaccard_output.sum() + jaccard_target.sum()

            loss -= self.jaccard_weight * torch.log((intersection + eps) / (union - intersection + eps))
        return loss

In [ ]:
"""
not used anymore
a = 0.5 / (224*224-20*30)
b = 0.5 / (20*30)
c = a+b
a /= c
b /= c
weights = [b,a]
class_weights = torch.FloatTensor(weights).cuda()
"""
detection_criterion = nn.CrossEntropyLoss()#weight=class_weights)#LossBinary(jaccard_weight=5)
# criterion = nn.NLLLoss()


def acc(predicted, labels):
  return (predicted == labels).sum().item()/labels.size(0)



def IoU(predicted, labels):
  inter = (predicted & labels).float().sum((1, 2))  
  union = (predicted | labels).float().sum((1, 2)) 
  iou = inter / union
  
  iou = float(iou.sum().item()/labels.size(0))
  
  return iou

def accuracy(net, test_loader, cuda=True, metric=acc):
  net.eval()
  correct = 0
  total = 0
  loss = 0
  with torch.no_grad():
      for data in test_loader:
          images, labels = data
          if cuda:
            images = images.type(torch.cuda.FloatTensor)
            labels = labels.type(torch.cuda.LongTensor)
          outputs = net(images)
          _, predicted = torch.max(outputs.data, 1)
          total += 1
          correct += metric(predicted, labels)

  net.train()
  print('Accuracy of the network on the test images: %d %%' % (
      100 * correct / (total+1)))
  # return (100.0 * correct / total, loss/total)
  return 100.0 * correct



def train(net, optimizer, criterion, train_loader, test_loader,  n_epoch = 5,
          train_acc_period = 1000,
          test_acc_period = 500,
          cuda=True,
          metric=acc):
  loss_train = []
  loss_test = []
  total = 0
  for epoch in range(n_epoch):  # loop over the dataset multiple times
      if epoch%4==1:
        torch.save(net,"net_satisfactoryB_epoch"+str(epoch)+".pt")
      print("epoch : ", epoch)
      running_loss = 0.0
      running_acc = 0.0
      for i, data in enumerate(train_loader, 0):
          # get the inputs
          inputs, labels = data
          
          if cuda:
            inputs = inputs.type(torch.cuda.FloatTensor)
            labels = labels.type(torch.cuda.LongTensor)
       
          # zero the parameter gradients
          optimizer.zero_grad()

   
          outputs = net(inputs)

          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()
          total += labels.size(0)
     
          running_loss = 0.33*loss.item() + 0.66*running_loss #/float(labels.size(0))


          _, predicted = torch.max(outputs.data, 1)
          correct = metric(predicted, labels)
       
          running_acc = 0.3*correct + 0.66*running_acc

          if i % train_acc_period == train_acc_period-1:
            print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss))
            print('[%d, %5d] acc: %.3f' %(epoch + 1, i + 1, running_acc))
            running_loss = 0.0
            total = 0
            # break
      if epoch % test_acc_period == test_acc_period-1:
          print('u')
          cur_acc = accuracy(net, test_loader, cuda=cuda, metric=metric)
          print('[%d] test acc: %.3f' %(epoch + 1, cur_acc))

  print('Finished Training')

In [ ]:
net = U_resNet()
#net = torch.load("net_loss0.001_ep12.pt")
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print("using cuda")
    net.cuda()
learning_rate = 1e-4
optimizer = torch.optim.Adam(net.parameters(),lr=learning_rate)
testloader = []
train(net,
      optimizer,
      detection_criterion,
      trainloader,
      testloader,
      n_epoch = 30,
      cuda=use_cuda,
      train_acc_period = 30,
      test_acc_period = 30,
      metric=IoU)

#accuracy(net, test_loader=testloader, cuda=use_cuda, metric=IoU)

detection_net = net

In [ ]:
accuracy(net, test_loader=testloader,cuda = use_cuda, metric=IoU)

In [ ]:
#torch.save(net,"net_satisfactory_epoch28.pt")

In [ ]:
fig=plt.figure(figsize=(64,64))
for i in range(5):
    #filename = trainfiles[56054]#30594
    #img = resize(load_img(filename),(224,224))
    I,M = trdata[5*i+4]
    mask = M[:,:,0]#masks_as_image(df.query('ImageId=="'+filename+'"')['EncodedPixels'])[:,:,0]
    img = I
    input = torch.Tensor(np.transpose(np.expand_dims(img,axis=0),(0, 3, 1, 2)))
    input = input.type(torch.cuda.FloatTensor)
    with torch.no_grad():
        lab = net(input)
    _, predicted = torch.max(lab.data, 1)

    l = predicted.type(torch.LongTensor).data.numpy()
    
    L = l[0]

    np.amax(mask)

    fig.add_subplot(5, 3,i*3+ 1)
    plt.imshow(img)
    fig.add_subplot(5, 3,i*3+ 2)
    plt.imshow(mask)
    fig.add_subplot(5, 3, i*3+3)
    plt.imshow(L)

# testing results

In [ ]:
net  = torch.load('net_satisfactory_epoch28.pt')

In [ ]:
tsdata = []

og = 768
s = 224
counter = 0
for filename in trainfiles[140000:148000] : 

  counter+=1
  if counter%1000 ==0:
        print("iteration ", counter)
  if df.query('ImageId=="'+filename+'"')['ship_count'][0] >=1 :
    im = load_img(filename)
    ma,centers = masks_as_image_crop(df.query('ImageId=="'+filename+'"')['EncodedPixels'])
    for k in range(len(centers)):
        j,i = centers[k]
        intersects = False
        for m,n in centers[:k]:
            if dist(j,i,m,n) <56 :
                intersects = True
        if not intersects :
            tsdata.append((im[max(i-s//2,0)-max(i+s//2-og,0):min(og,i+s//2) + max(0,s//2-i),
                          max(j-s//2,0)-max(j+s//2-og,0):min(og,j+s//2) + max(0,s//2-j)],
                       ma[max(i-s//2,0)-max(i+s//2-og,0):min(og,i+s//2) + max(0,s//2-i),
                          max(j-s//2,0)-max(j+s//2-og,0):min(og,j+s//2) + max(0,s//2-j)]))

In [ ]:
testloader = prepare_dataloader(*prepare_dataset(tsdata),16)

In [ ]:
use_cuda = True
for i in range(4,29,4):
    net  = torch.load(f'net_satisfactory_epoch{i}.pt')
    print("epoch ",i)
    accuracy(net, test_loader=testloader, cuda=use_cuda, metric=IoU)

In [ ]:
l = [0,36,50,49,55,53,58,55]
plt.plot(range(0,29,4),l)
plt.xlabel("epoch")
plt.ylabel("success rate")

In [ ]:

load_imgT = lambda filename: np.array(PIL.Image.open(f"test_v2/{filename}"))
tsdata = []

og = 768
s = 224
counter = 0
for filename in testfiles[2:3] : 

    counter+=1
    if counter%1000 ==0:
        print("iteration ", counter)
  
    im = load_imgT(filename)
    #plt.imshow(im)
    tsdata.append(im)
    MASK = np.zeros((og,og))
    for k in range(4):
        for l in range(4):
            img = im[224*k - 43*k: 224*(k+1)-43*k, 224*l - 43*l: 224*(l+1) - 43*l]
            #tsdata.append(img)
            input = torch.Tensor(np.transpose(np.expand_dims(img,axis=0),(0, 3, 1, 2)))
            input = input.type(torch.cuda.FloatTensor)
            with torch.no_grad():
                lab = net(input)
            _, predicted = torch.max(lab.data, 1)

            L = predicted.type(torch.LongTensor).data.numpy()
            
            MASK[224*k - 43*k: 224*(k+1)-43*k, 224*l - 43*l: 224*(l+1) - 43*l] += L[0]
    MASK = MASK//2
    plt.imshow(MASK,cmap='gray')
        
            
    

In [ ]:
plt.imshow(tsdata[0])

In [ ]:
fig=plt.figure(figsize=(64,64))
for i in range(16):
    #filename = trainfiles[56054]#30594
    #img = resize(load_img(filename),(224,224))
    img = tsdata[5*i+4]
   
    
    input = torch.Tensor(np.transpose(np.expand_dims(img,axis=0),(0, 3, 1, 2)))
    input = input.type(torch.cuda.FloatTensor)
    with torch.no_grad():
        lab = net(input)
    _, predicted = torch.max(lab.data, 1)

    l = predicted.type(torch.LongTensor).data.numpy()
    
    L = l[0]

   

    fig.add_subplot(5, '2',i*2+ 1)
    plt.imshow(img)
    fig.add_subplot(5, 2, i*2+2)
    plt.imshow(L)

# Alexnet header

In [ ]:
class AlexNet(nn.Module):
    
    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
    )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        ## Mettre un max-pooling 
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x
    
def alexnet(num_classes, pretrained=False, **kwargs):
    model = AlexNet(1000,**kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['alexnet']))  ### checker comment se fait la copie
        for p in model.features.parameters():
            p.requires_grad=False
    model.classifier=AlexNet(2,**kwargs).classifier
    return model

In [ ]:
alex = torch.load("classifier_epoch7.pt")

In [ ]:
trdata = []
Testdata = []
og = 768
s = 224
counter1 = 0
counter2=0
nber_files=15000
count1 = 0
count2 = 0
   
for filename in trainfiles[nber_files:nber_files + 10000]:
    im = load_img(filename)
    
    
    
    if df.query('ImageId=="'+filename+'"')['ship_count'][0] >=1:
        count1+=1
        Testdata.append((torch.Tensor(im).permute(2,0,1),1))
        if count1>=1500 :
            break
    else :
        if count2>=1500 :
            if count1%100 ==0 :
                print("count2 acheived")
                print("count 1 is ", count1)
            continue
        count2+=1
        Testdata.append((torch.Tensor(im).permute(2,0,1),0))
    
    
    
    



In [ ]:
testloader=torch.utils.data.DataLoader(Testdata,22)

In [ ]:
cuda = True
def accuracyMC(net, test_loader, cuda=True):
  net.eval()
  correct = 0
  total = 0
  loss = 0
  confusion_matrix = [[0,0],[0,0]]
  with torch.no_grad():
      for data in test_loader:
          images, labels = data
          
          if cuda:
            images = images.type(torch.cuda.FloatTensor)
            labels = labels.type(torch.cuda.LongTensor)
          outputs = net(images)
          # loss+= criterion(outputs, labels).item()
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          for i in range(labels.size(0)):
            confusion_matrix[labels[i].item()][predicted[i].item()] +=1
          correct += (predicted == labels).sum().item()
          
  print(confusion_matrix)
  return 100.0 * correct/total

In [ ]:
accuracyMC(alex,testloader)